In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import math
from scipy.fft import fft2, ifft2
import time
from time import strftime, gmtime
import numba
from numba import jit
import pandas as pd
import sys

import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()

from timeit import default_timer as timer

import sympy

font = {'family': 'serif',
          'color':  'black',
          'weight': 'normal',
          'size': 16,
          }

In [15]:
global first_term, second_term, third_term, V_ph, V_ph_k, rho, dt, U

N = 2*2048
k_max = N
L = 40

#del r_discretization, k_discretization
dl = L / N
l = np.linspace(-L,L,N)
X, Y = np.meshgrid(l,l)

k2 = np.zeros(N)
p = 2*np.pi/L
for i in range(0,N/2):
  k2 = pow(i*p,2) 
for i in range(N/2,N):
  k2 = pow((i-1)*p,2)

Kx, Ky = np.meshgrid(k2,k2)

K2 = Kx*Kx+Ky*Ky
K = np.sqrt(K2)

g = np.ones((N,N), dtype=np.complex64)
S = np.ones((N,N), dtype=np.complex64)

#Initializing the terms
first_term = np.zeros((N,N), dtype=np.complex64)
second_term = np.zeros((N,N), dtype=np.complex64)
third_term = np.zeros((N,N), dtype=np.complex64)

V_ph = np.zeros((N,N), dtype=np.complex64) 
V_ph_k = np.zeros((N,N), dtype=np.complex64)

v = U*np.exp(-X*X-Y*Y)

In [12]:
U = 10
condition = True
while condition:
  
  S = 1 + rho * fft2(g-1)
  omega_k = -0.5*K2*(2*S+1)*(1-(1/S))*(1-(1/S))
  omega = ifft2(omega_k)
  V_ph = g*v+2*gradient_g+(omega-1)*g
  V_ph_k = fft2(V_ph)
  dS = K / np.sqrt(K2+2*V_ph_k)
  S = S + (1-dt)*dS

In [19]:
np.sqrt(2)

1.4142135623730951

In [17]:
g

array([[1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       ...,
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j]],
      dtype=complex64)

In [18]:
1/g

array([[1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       ...,
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j]],
      dtype=complex64)